# Electrolyte Optimiser (PumpBot V2)

Update experiment details in [yaml file](config/hardware_config.yaml). Then run the below code block **once per session**.

In [1]:
%load_ext autoreload
%autoreload 2

from src.scheduler import scheduler
config_path = "config/hardware_config.yaml"

device = scheduler(config_path)

INFO:root:Configuring pump controller serial port..
INFO:root:Attempting to open pump controller serial port..
INFO:root:Simulated connection to pump controller established.
INFO:root:Simulated connection to pump controller established.
INFO:root:Temperature regulator PID mode successfully configured.
INFO:root:Temperature regulator dead band settings successfully configured.
INFO:root:Temperature regulator voltage alarm settings successfully configured.
INFO:root:Temperature regulator current alarm settings successfully configured.
INFO:root:Temperature Sensor #1 successfully configured.
INFO:root:Temperature sensor #2 successfully configured.
INFO:root:Successfully updated steinhart coefficients for temperature sensor #1.
INFO:root:Successfully updated steinhart coefficients for temperature sensor #2.
INFO:root:Temperature regulator fan settings successfully configured.
INFO:root:Resetting AC and DC dataframes..


## Useful functions..

Hover over functions for more information.

In [ ]:
device.ensure_primed()

In [ ]:
device.deprime_lines()

In [ ]:
device.system_flush()

## Run the next code block to perform a single experiment.

In [2]:
device.run_basic_experiment(deprime=False)

INFO:root:Temperature regulator set to cooling mode.
INFO:root:Peltier target temperature set to 0.0C.
INFO:src.scheduler:Priming lines first..
INFO:src.scheduler:Dosing Milli-Q: 0.200 ml on A[1]
INFO:src.scheduler:Primed Milli-Q with 0.2ml.
INFO:src.scheduler:Transferring 2.4ml to cell..
INFO:src.scheduler:Transferring 2.4ml to waste #1..
INFO:src.scheduler:Dosing Ethanol: 0.200 ml on A[2]
INFO:src.scheduler:Primed Ethanol with 0.2ml.
INFO:src.scheduler:Transferring 2.4ml to cell..
INFO:src.scheduler:Transferring 2.4ml to waste #1..
INFO:src.scheduler:Dosing Chem3: 0.200 ml on A[3]
INFO:src.scheduler:Primed Chem3 with 0.2ml.
INFO:src.scheduler:Transferring 2.4ml to cell..
INFO:src.scheduler:Transferring 2.4ml to waste #1..
INFO:src.scheduler:Dosing Chem4: 0.200 ml on A[4]
INFO:src.scheduler:Primed Chem4 with 0.2ml.
INFO:src.scheduler:Transferring 2.4ml to cell..
INFO:src.scheduler:Transferring 2.4ml to waste #1..
INFO:src.scheduler:Dosing Chem5: 2.000 ml on B[1]
INFO:src.scheduler:Pri

## Atinary Campaign

In [1]:
from sdlabs_wrapper.wrapper import initialize_optimization
import json

optimiser_config = "config/conductivity_optimiser.json"

In [ ]:
# load config as dict
with open(optimiser_config, "rb") as f:
    config_dict = json.load(f)

wrapper = initialize_optimization( # API key given as env variable SDLABS_API_KEY
    spec_file_content=config_dict,
    inherit_data=False, 
    always_restart=False, # Change to true to restart
)

for iteration in range(wrapper.config.budget):

    print(f"Iteration {iteration+1}: Fetching new suggestions..")

    # Atinary will return suggestions until measurements received - useful in case of resume
    suggestions = wrapper.get_new_suggestions(max_retries=10, sleep_time_s=30)

    if not suggestions:
        raise RuntimeError(f"No suggestions received on iteration {iteration+1}.")
    
    for suggestion in suggestions:
            print(f"New suggestion received for iteration {iteration+1}: {suggestion.param_values}.")

            # Update yaml with suggestions
            device.update_yaml_volumes(suggestion.param_values)
            
            # Calculate cost of new mixture
            cost = device.calculate_cost()

            # Synthesise and analyse at target_temp
            device.run_basic_experiment()

            # Build table of measurements to send e.g. [conductivity, cost]
            results = [1.0, 0.0]

            for i, obj in enumerate(wrapper.config.objectives):
                # e.g. {'conductivity': 0.06925926902246848, 'cost': 0.9500057653400364}
                suggestion.measurements[obj.name] = results[i] # Send data here

            wrapper.send_measurements(suggestions)
            print(f"Iteration {iteration+1} measurements sent.")

            # Clean test cell whilst optimiser calculates next suggestions
            device.system_flush()